<a href="https://colab.research.google.com/github/ucfilho/marquesgabi/blob/master/Calculo_018_out_31_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# carrega as bibliotecas do python

In [0]:
%matplotlib inline
from matplotlib import pyplot as plt
import os 
import cv2
import zipfile
from skimage import data
from skimage import filters
from skimage import exposure
import skimage.feature as sk
import numpy as np
import pandas as pd
from google.colab import files
from sklearn.metrics.cluster import entropy
from sklearn.metrics.cluster import homogeneity_score
from numpy import linalg as LA


In [3]:
!wget http://ftp.feq.ufu.br/ubirajara/Sugar_set_30_2019.zip
#!ls -l trash_nov_22_2018.zip


--2019-11-01 02:52:55--  http://ftp.feq.ufu.br/ubirajara/Sugar_set_30_2019.zip
Resolving ftp.feq.ufu.br (ftp.feq.ufu.br)... 200.19.144.11
Connecting to ftp.feq.ufu.br (ftp.feq.ufu.br)|200.19.144.11|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19584173 (19M) [application/zip]
Saving to: ‘Sugar_set_30_2019.zip.13’

Sugar_set_30_2019.z 100%[===================>]  18.68M  5.56MB/s    in 3.4s    

2019-11-01 02:52:59 (5.56 MB/s) - ‘Sugar_set_30_2019.zip.13’ saved [19584173/19584173]



In [0]:

file_name = zipfile.ZipFile('Sugar_set_30_2019.zip', 'r')
file_name.extractall() 

In [0]:
# all images are resized in variable x, are in gray in variable p and in variable w are dataframe
FILE='Sugar_set_30_2019.zip'
img_name=[]
x=[]
p=[]
w=[]
Size=5 # trocar por 64 qdo acabar a analise

with zipfile.ZipFile(FILE, "r") as f:
    for name in f.namelist():
        img_name.append(name)
        #x.append(cv2.imread(name))
        x.append(cv2.resize(cv2.imread(name),(Size,Size)))

nrow=len(img_name)
ncol=Size*Size
pp=np.zeros((nrow,ncol))
for i in range(nrow):
    w.append(cv2.cvtColor(np.array(x[i]), cv2.COLOR_BGR2GRAY))
    pp[i,:]=w[i].ravel()

In [0]:
Nfotos=len(img_name)
GLCM=np.zeros((Nfotos, 14))   
Nrow=Size
Ncol=Size
#print(GLCM.shape)
#Soma=np.sum(pp[k,:])

G=Ncol-1

for k in range(Nfotos):
  Pdif=[] 
  Psom=[]
  p=pp[k,:].reshape(Size,Size)
  Soma=0
  px=[]
  py=[]
  for i in range(Size):
    for j in range(Size):
      Soma=Soma+p[i,j]
  
  for i in range(Size):
    for j in range(Size):
      p[i,j]=p[i,j]/Soma

  for i in range(Size):
    Somax=0
    for j in range(Size):
      Somax=Somax+p[i,j]
    px.append(Somax)

  for j in range(Size):
    Somay=0
    for i in range(Size):
      Somay=Somay+p[i,j]
    py.append(Somay)

  for kr in range(2*G+1):
    Soma_soma=0
    for i in range(Nrow):
      for j in range(Ncol):
        if((i+j)==kr):
          Soma_soma=Soma_soma+p[i,j]
    Psom.append(Soma_soma)

  for kr in range(G+1):
    Soma_dif=0
    for i in range(Nrow):
      for j in range(Ncol):
        if((abs(i-j))==kr):
          Soma_dif=Soma_dif+p[i,j]
    Pdif.append(Soma_dif)

  HXY=0
  HXY1=0
  HXY2=0
  for i in range(Nrow):
    for j in range(Ncol):
      HXY=HXY-p[i,j]*np.log(p[i,j])
      HXY1=HXY1-p[i,j]*np.log(px[i]*py[j])
      HXY2=HXY2-px[i]*py[j]*np.log(px[i]*py[j])
  HX=0
  for i in range(Nrow):
    HX=HX-px[i]*np.log(px[i])
  HY=0
  for j in range(Nrow):
    HY=HY-py[j]*np.log(py[j])
  if(HX>HY):
    maxH=HX
  else:
    maxH=HY

  for i in range(Nrow):
    for j in range(Ncol):
      #print("i=%d j=%d k=%d"%(i,j,k))
      GLCM[k,0]=GLCM[k,0]+p[i,j]**2 #segundo momento angular
      GLCM[k,1]=GLCM[k,1]+(i-j)**2*p[i,j] #contraste


      MIx=0;MIy=0
      for ii in range(Nrow):
        for jj in range(Ncol):
          MIy=MIy+jj*p[ii,jj]
          MIx=MIx+ii*p[ii,jj]
      Sx2=0;Sy2=0;
      for ii in range(Nrow):
        for jj in range(Ncol):
          Sx2=Sx2+(ii-MIx)**2*p[ii,jj]
          Sy2=Sy2+(jj-MIy)**2*p[ii,jj]
      Sx=Sx2**0.5
      Sy=Sy2**0.5


      GLCM[k,2]=GLCM[k,2]+((i*j)*p[i,j]-MIx*MIy)/(Sx*Sy) #Correlacao
      # CONFERIR CORRELACAO EM OUTROS

      GLCM[k,3]=GLCM[k,3]+(i-np.mean(p))**2*p[i,j] #Variancia

      GLCM[k,4]=GLCM[k,4]+(1/(1+(i-j)**2))*p[i,j]#INVERSA DIF MOMENTO
      
    f8=0
      
    for kr in range(len(Psom)):
      GLCM[k,5]=GLCM[k,5]+kr*Psom[kr] # conferir !!!!! ok???
      f8=f8-Psom[kr]*np.log(Psom[kr])

    for kr in range(len(Psom)):
      GLCM[k,6]=GLCM[k,6]+(kr-f8)**2*Psom[kr] # sum of variance

    GLCM[k,7]=f8  # sum entropy
    GLCM[k,8]=GLCM[k,8]-p[i,j]*np.log(p[i,j]) # entropia

    for kr in range(len(Pdif)):
      GLCM[k,9]=GLCM[k,9]+(kr)**2*Pdif[kr] # variance difference
      GLCM[k,10]=GLCM[k,10]-Pdif[kr]*np.log(Pdif[kr]) # difference entropy
      
    GLCM[k,11]=(HXY-HXY1)/maxH # information measure of correlation 1
    GLCM[k,12]=(1-np.exp(-2*(HXY2-HXY)))**0.5 # information measure of correlation 2

    Q=np.zeros((Nrow,Ncol))
    for i in range(Nrow):
      for j in range(Ncol):
        for kr in range(Ncol):
          Q[i,j]=Q[i,j]+p[i,kr]*p[j,kr]/(px[i]*py[kr])
    
    #v = LA.eig(Q)
    v=LA.eigvals(Q)
    n=len(v)
    v.sort()
    #print(v)
    #print(v[n-2])
    GLCM[k,13]=v[n-2]**0.5 # maximal correlation coefficient 

In [0]:
# ver os conceitos GLCM aqui tambem
# http://haralick.org/journals/TexturalFeatures.pdf

In [12]:
pd.set_option('display.max_columns', 30)
print(pd.DataFrame(GLCM))

           0         1          2         3         4          5          6  \
0   0.040159  4.006764 -48.388893  5.873156  0.423837  20.097900  38.848134   
1   0.041540  4.282528 -47.724904  5.762803  0.411868  20.118283  38.570089   
2   0.040287  4.030399 -49.193342  5.979967  0.424910  20.208554  39.133866   
3   0.040175  4.015663 -48.607804  6.027608  0.435603  20.231465  39.597120   
4   0.040313  4.043838 -47.746904  5.656749  0.422210  19.935322  38.085237   
5   0.040234  4.056964 -49.752577  5.948915  0.431240  20.434179  40.230847   
6   0.040177  3.895373 -49.375098  5.944445  0.435292  20.236533  39.573951   
7   0.040276  3.931919 -51.702480  6.036687  0.428104  20.603832  40.693982   
8   0.040229  3.993338 -49.941784  6.066993  0.427480  20.420368  40.181363   
9   0.040272  3.979461 -49.007498  5.888981  0.430643  20.247112  39.592274   
10  0.040134  3.934069 -48.671268  5.729726  0.432728  20.036278  38.554257   
11  0.041463  3.928237 -48.569188  5.890711  0.44786

In [13]:
GLCM=pd.DataFrame(GLCM)
pd.set_option('display.max_columns', 30)
print(GLCM.head(5))

          0         1          2         3         4          5          6  \
0  0.040159  4.006764 -48.388893  5.873156  0.423837  20.097900  38.848134   
1  0.041540  4.282528 -47.724904  5.762803  0.411868  20.118283  38.570089   
2  0.040287  4.030399 -49.193342  5.979967  0.424910  20.208554  39.133866   
3  0.040175  4.015663 -48.607804  6.027608  0.435603  20.231465  39.597120   
4  0.040313  4.043838 -47.746904  5.656749  0.422210  19.935322  38.085237   

          7         8          9        10        11        12        13  
0  2.080038  0.648663  20.033820  7.608990 -0.000770  0.049748  0.039787  
1  2.064334  0.646616  21.412639  7.688757 -0.005531  0.132720  0.133372  
2  2.071191  0.656965  20.151997  7.628194 -0.001239  0.063068  0.045709  
3  2.083742  0.653877  20.078315  7.654684 -0.000658  0.045987  0.036992  
4  2.073301  0.652921  20.219188  7.614789 -0.001073  0.058718  0.051329  


In [14]:
import numpy as np
#from skimage.feature import haar_like_feature_coord
#from skimage.feature import draw_haar_like_feature
from skimage.feature import greycomatrix
from skimage.feature import greycoprops
#import skimage.feature.greycoprops as greycoprops
image = np.array([[0, 0, 1, 1],[0, 0, 1, 1],[0, 2, 2, 2],[2, 2, 3, 3]], dtype=np.uint8)
g = greycomatrix(image, [1, 2], [0, np.pi/2], levels=6,normed=True, symmetric=True)
contrast = greycoprops(g, 'contrast')
correlation= greycoprops(g, 'correlation')
print(contrast)
print(correlation)

[[0.58333333 1.        ]
 [1.25       2.75      ]]
[[ 0.71953255  0.48571429]
 [ 0.41176471 -0.29411765]]
